# Table 1

**Table 1. Area and mean EKE over each region of study.** This table shows the total area of each study region (Area) and the area-weighted mean of the temporally averaged EKE computed over the period 1993-2022 from the all-sat and two-sat altimetric products (Mean EKE). The regions analyzed include the global ocean (Global), the high EKE regions (High EKE), the tropical band (Tropical), the global ocean excluding the high EKE regions and the tropical band (Residual), and each specific high EKE region: Gulf Stream (GS), Kuroshio Extension (KE), Agulhas Current (AC), Brazil Malvinas Confluence region (BMC), Loop Current (LC), Great Whirl and Socotra Eddy in East Africa (GWSE), and East Australian Current (EAC).

### Import libraries

In [1]:
%reload_ext autoreload
%autoreload 2

import xarray as xr
import numpy as np
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
import utils

### Open masks

In [2]:
masks = xr.open_dataset('../../results/masks.nc')

### Open files with the EKE mean

In [3]:
sats = ['ALL_SAT', 'TWO_SAT']
dic = {sat: {} for sat in sats}
for sat in sats:

    path_tmean = f'../../results/EKE_{sat}/tmean/EKE_{sat}_tmean.nc'

    dic[sat]['tmean'] = xr.open_dataset(path_tmean)
    

### Compute global area and save it into an xarray dataset

In [4]:
area            = utils.get_area_haversine(masks.lat, masks.lon) # (km^2)
da_area         = xr.Dataset(coords={'lat': masks.lat, 'lon': masks.lon})
da_area['area'] = (('lat', 'lon'), area)

### Parameters to create the table

In [5]:
# Set the display format for floating-point numbers
pd.options.display.float_format = "{:.2f}".format

regions = [
    "Global",
    "High EKE",
    "Tropical",
    "Residual",
    "GWSE",
    "AC",
    "KE",
    "GS",
    "LC",
    "BMC",
    "EAC",
]

regions_ = [
    "no ice",
    "high eke",
    "tropical",
    "remainder",
    "GWSE",
    "AC",
    "KE",
    "GS",
    "LC",
    "BMC",
    "EAC",
]


label_unit = "cm$^2$ s$^{-2}$"

metrics = [
    "Area (x10$^6$ km$^2$)",
    f"Mean EKE ({label_unit})",

]

### Create the table

In [6]:

columns = pd.MultiIndex.from_product([metrics, ["ALL SAT", "TWO SAT"]])
df = pd.DataFrame(index=regions, columns=columns)

for j, region in enumerate(regions):
    reg = regions_[j]
    
    for k, sat in enumerate(["ALL SAT", "TWO SAT"]):
        sat_ = sat.replace(" ", "_")
        
        # ----- Area -----
        df.loc[region, ("Area (x10$^6$ km$^2$)", sat)] = (
            masks[reg] * da_area['area']).sum().values / 1e6
        
    
        # ----- Mean EKE -----
        # area-weighted mean EKE 
        df.loc[region, (f"Mean EKE ({label_unit})", sat)] = (
            np.float32(utils.get_mean_haversine(dic[sat_]['tmean']['EKE'], masks[reg])))
    
    
# convert dataframe to LaTeX
latex_table = df.to_latex('../../tables/table_1.tex')
csv_table   = df.to_csv('../../tables/table_1.csv')

df

/var/folders/1r/g9d70q7x59s6sd00xl3mmyfw0000gn/T/ipykernel_29618/3992270281.py:22: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_table = df.to_latex('../../tables/table_1.tex')


Area (x10$^6$ km$^2$)         Mean EKE (cm$^2$ s$^{-2}$)        
                       ALL SAT TWO SAT                    ALL SAT TWO SAT
Global                  327.89  327.89                     224.83  227.09
High EKE                 16.54   16.54                    1018.12 1003.62
Tropical                 43.94   43.94                     330.51  354.68
Residual                277.34  277.34                     152.73  152.46
GWSE                      1.90    1.90                    1179.16 1133.10
AC                        5.92    5.92                     982.23  974.27
KE                        2.72    2.72                    1043.04 1028.11
GS                        2.80    2.80                    1078.97 1061.14
LC                        0.51    0.51                    1133.63 1124.19
BMC                       1.56    1.56                     838.23  837.05
EAC                       1.12    1.12                     921.61  914.49